In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import re
from sklearn import preprocessing
#from dirty_cat import SimilarityEncoder
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string

C:\Users\devquinn\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Data Preprocessing

In [2]:

def calc_old2new_similarity(sources, targets):

    similarities = []
   
    for i in range(len(sources)):
        similarity = fuzz.token_sort_ratio(sources[i], targets[i])/100
        similarities.append(similarity)

    return similarities

In [3]:
path = 'C:/Users/devquinn/OneDrive - Deloitte (O365D)/Desktop/atom-dc-coa/'

## Company: Fanatics

In [4]:
file = 'Fanatics/GL_Account_Mapping_V5.11.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="GL Account Mapping", dtype={'Source Account #': str, 'Target Account #': str}, header=0)
df.head(3)

,Account Type,Source Level 1,Source Level 2,Source Level 3,Source Management P&L,Source Account #,Source Account Description,Target Level 1,Target Level 2,Target Level 3,Target Management PnL,Target Account #,Target Account Description,Unnamed: 13
0,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100110,Petty Cash,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100001,Petty Cash,NaN
1,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100111,Petty Cash - Dreams,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100001,Petty Cash,NaN
2,Balancesheet,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100120,Store Fund - Avenues,Assets,Current Assets,Cash and Cash Equivalents,Cash and Cash Equivalents,100003,Store Fund - Avenues,NaN


In [5]:
cols = [5,1,2,3,6,11,7,8,9,12]
df_fan = df[df.columns[cols]].dropna()
df_fan = df_fan.rename(columns = {"Source Level 1": "S L1", "Source Level 2": "S L2", "Source Level 3": "S L3", "Source Account Description": "Source Description",
                                  "Target Level 1": "T L1", "Target Level 2": "T L2", "Target Level 3": "T L3", "Target Account Description": "Target Description"})

sources = df_fan['Source Description'].tolist()
targets = df_fan['Target Description'].tolist()
similarities = calc_old2new_similarity(sources, targets)
df_fan = df_fan.assign(Similarity = similarities)
df_fan.head(3)

,Source Account #,S L1,S L2,S L3,Source Description,Target Account #,T L1,T L2,T L3,Target Description,Similarity
0,100110,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash,100001,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash,1.00
1,100111,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash - Dreams,100001,Assets,Current Assets,Cash and Cash Equivalents,Petty Cash,0.74
2,100120,Assets,Current Assets,Cash and Cash Equivalents,Store Fund - Avenues,100003,Assets,Current Assets,Cash and Cash Equivalents,Store Fund - Avenues,1.00


## Company: Mars

In [6]:
file = "Mars/Mars Data.xlsx"
df_map = pd.read_excel(open(path+file, 'rb'), sheet_name="Mapping", dtype = str, header=0)
df_target = pd.read_excel(open(path+file, 'rb'), sheet_name="Target COA", dtype = str, header=0)
df_source1 = pd.read_excel(open(path+file, 'rb'), sheet_name="Source - Atlas ERP", dtype = str, header=0)
df_source2 = pd.read_excel(open(path+file, 'rb'), sheet_name="Source - P12 ERP", dtype = str, header=0)

In [7]:
df_map.head(1)

,ERP,Source GL Account,Source GL Account Description,Target GL Account,Target GL Account Description
0,Atlas,110000,Sales at List Price (Auto),400000,Gross Sales Products - 3rd Party Domestic - Auto


In [8]:
df_mars = pd.DataFrame()
sources_not_found = []
targets_not_found = []

for i in range(len(df_map)):

    source = df_map.loc[i, 'Source GL Account']
    source_description = df_map.loc[i, 'Source GL Account Description']
    source_row = df_source1.where(df_source1['Account'] == source).dropna(how='all')
    if len(source_row) == 0:
        source_row = df_source2.where(df_source2['Account'] == source).dropna(how='all')
        if len(source_row) == 0:
            sources_not_found.append(source)
            continue
    source_hierarchy = source_row.iloc[0][1:].tolist()

    target = df_map.loc[i, 'Target GL Account']
    target_description = df_map.loc[i, 'Target GL Account Description']
    target_row = df_target.where(df_target['Account'] == target).dropna(how='all')
    if len(target_row) == 0:
        targets_not_found.append(target)
        continue
    target_hierarchy = target_row.iloc[0][:4].tolist()

    df_mars = df_mars.append({'Source Account #': source, 'S L1': source_hierarchy[0], 
                            'S L2': source_hierarchy[1], 'S L3': source_hierarchy[2],
                            'S L4': source_hierarchy[3], 'S L5': source_hierarchy[4],
                            'S L6': source_hierarchy[5], 'S L7': source_hierarchy[6],
                            'S L8': source_hierarchy[7], 'S L9': source_hierarchy[8],
                            'S L10': source_hierarchy[9], 'Source Description': source_description,

                            'Target Account #': target, 'T L1': target_hierarchy[0], 
                            'T L2': target_hierarchy[1], 'T L3': target_hierarchy[2],
                            'T L4': target_hierarchy[3], 'Target Description': target_description,
                            }, ignore_index=True)

if len(sources_not_found) > 0 or len(targets_not_found) > 0:
    print(f'Error: some sources/targets from mapping sheet not found in any hierarchy sheets. These entries have been dropped')

df_mars.head(3)

Error: some sources/targets from mapping sheet not found in any hierarchy sheets. These entries have been dropped


,Source Account #,S L1,S L2,S L3,S L4,S L5,S L6,S L7,S L8,S L9,S L10,Source Description,Target Account #,T L1,T L2,T L3,T L4,Target Description
0,110000,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
1,110005,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price - BIFG (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto
2,110010,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price Adjustment - Manual,400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto


In [9]:
print(len(sources_not_found), len(targets_not_found))

274 6


In [10]:
sources_not_found[0]

'172045'

In [11]:
sources_dfmap = set(df_map['Source GL Account'])
targets_dfmap = set(df_map['Target GL Account'])
sources_dfsource = set(df_source1['Account']).union(set(df_source2['Account']))
targets_dftarget = set(df_target['Account'])

#print(len(sources_dfmap), len(targets_dfmap), len(sources_dfsource), len(targets_dftarget))

print(len(sources_dfmap.difference(sources_dfsource)), len(targets_dfmap.difference(targets_dftarget)))

274 7


In [12]:
sources = df_mars['Source Description'].tolist()
targets = df_mars['Target Description'].tolist()

similarities = calc_old2new_similarity(sources, targets)
df_mars = df_mars.assign(Similarity = similarities)
df_mars.head(3)

,Source Account #,S L1,S L2,S L3,S L4,S L5,S L6,S L7,S L8,S L9,S L10,Source Description,Target Account #,T L1,T L2,T L3,T L4,Target Description,Similarity
0,110000,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto,0.53
1,110005,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price - BIFG (Auto),400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto,0.49
2,110010,M003 Mars Financial Statement V2 (Full Trial B...,Profit and Loss,Operating Profit,6. Total Net Sales,3. Net Sales - third party domestic,Gross Sales - Domestic,NaN,NaN,NaN,NaN,Sales at List Price Adjustment - Manual,400000,Revenue & Trade Expenditures,Revenue - 3rd Party,Revenue - Products - 3rd Party,Revenue - Products - 3rd Party Domestic,Gross Sales Products - 3rd Party Domestic - Auto,0.37


## Company: Peleton

In [13]:
file = 'Peleton/GL_Mapping1.20.xlsx'
df_map = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_map = df_map[df_map.columns[[0, 3]]]

file = 'Peleton/GL_SourceHierarchy1.20.xlsx'
df_source = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_source = df_source[df_source.columns[[2, 3, 4]]]

file = 'Peleton/GL_TargetHierarchy1.22.xlsx'
df_target = pd.read_excel(open(path+file, 'rb'), sheet_name="Sheet1", dtype=str, header=0)
df_target = df_target[df_target.columns[[2, 3, 4]]]

df_map.head(3)

,S1,T1
0,10000,100120
1,10001,100080
2,10004,100100


In [14]:
df_pel = pd.DataFrame()

for i in range(len(df_map)):
    #skip entries with empty sources/targets
    if any(df_map.iloc[i,:].isna()): continue
    #extract source and target numbers from mapping df
    source = df_map.loc[i, 'S1']
    target = df_map.loc[i, 'T1']

    if source not in df_source['account'].values:
        print(f'Error: Source {source} from mapping sheet not found in source sheet')
        continue
    else:
        #extract description of source number from source sheet 
        source_description = df_source.where(df_source['account'] == source).dropna(how='all')['Descriptions'].item()
        #extract source hierarchy label
        source_level = df_source.where(df_source['account'] == source).dropna(how='all')['K3'].item()

    if target not in df_target['account'].values:
        print(f'Error: Target {target} from mapping sheet not found in target sheet')
        continue
    else:
        find_target = df_target.where(df_target['account'] == target).dropna(how='all')
        if len(find_target) > 1:
            print(f'Error: target {target} maps to more than one account description')
            continue
        else:
            #extract description of target number from target sheet
            target_description = find_target['Descriptions'].item()
            #extract target hierarchy label
            target_level = find_target['K3'].item()

        
    df_pel = df_pel.append({'Source Account #': source, 'S L1': source_level, 'Source Description': source_description,
                            'Target Account #': target, 'T L1': target_level, 'Target Description': target_description,
                            }, ignore_index=True)
    
sources = df_pel['Source Description'].tolist()
targets = df_pel['Target Description'].tolist()

similarities = calc_old2new_similarity(sources, targets)
df_pel = df_pel.assign(Similarity = similarities)

df_pel.head(3)

Error: Target 210553 from mapping sheet not found in target sheet
Error: Target 210553 from mapping sheet not found in target sheet
Error: Target 210553 from mapping sheet not found in target sheet
Error: target 402007 maps to more than one account description
Error: target 509000 maps to more than one account description
Error: target 509001 maps to more than one account description
Error: target 509002 maps to more than one account description
Error: target 509000 maps to more than one account description
Error: target 509001 maps to more than one account description
Error: target 509002 maps to more than one account description
Error: target 601306 maps to more than one account description
Error: target 601306 maps to more than one account description
Error: target 590031 maps to more than one account description


,Source Account #,S L1,Source Description,Target Account #,T L1,Target Description,Similarity
0,10000,Cash,Cash and Cash Equivalents,100120,Cash and Cash Equivalents,Cash (Parent: Legacy),0.47
1,10001,Cash,FR Bank Operating - 7965,100080,Cash and Cash Equivalents,First Republic Bank Operating: 7965: Main,0.72
2,10004,Cash,FR Bank - Payroll - 2978,100100,Cash and Cash Equivalents,FR Bank: Payroll: 2978,1.00


## Company: Republic National Dist

In [42]:
file = 'Republic National Dist/Future State Mapping Consolidated Deliverable.xlsx'
df = pd.read_excel(open(path+file, 'rb'), sheet_name="Account", dtype=str, header=3)
df.head(3)

,Unnamed: 0,MasterMap,Company,AS400,Young's Oracle EBS Code,Young's Oracle EBS Description,RNDC Oracle EBS Account Code,RNDC Oracle EBS Sub-Account,RNDC Oracle EBS Account Description,Account Number Range,Account Type,General Account Number Range,General Account Classification,Specific Account Number Range,Specific Account Classification,Account Code,Account Name,Change Log/Comments
0,NaN,1000000000,AS400,1000000000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN
1,NaN,1000100000,AS400,1000100000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN
2,NaN,1000990000,AS400,1000990000,110100,MAIN CONCENTRATION,10020,**,NaN,100000-199999,Assets,100000-109999,Cash,101100-104999,Cash,101101,Cash - Master Cash,NaN


In [48]:
company = "Young's"
df_youngs = df.query(f'Company == "{company}"')

data_youngs = { 'Source Account #': df_youngs["Young's Oracle EBS Code"].dropna(),
                'S L1': '*MISSING*',
                'Source Description': df_youngs["Young's Oracle EBS Description"].dropna(),
                'Target Account #': df_youngs['Account Code'].dropna(),
                'T L1': df_youngs['Account Type'].dropna(),
                'T L2': df_youngs['General Account Classification'].dropna(),
                'T L3': df_youngs['Specific Account Classification'].dropna(),
                'Target Description': df_youngs['Account Name'].dropna()}

df_rnd = pd.DataFrame(data_youngs)

sources = df_rnd['Source Description'].tolist()
targets = df_rnd['Target Description'].tolist()

similarities = calc_old2new_similarity(sources, targets)
df_rnd = df_rnd.assign(Similarity = similarities)

df_rnd.head(3)

,Source Account #,S L1,Source Description,Target Account #,T L1,T L2,T L3,Target Description,Similarity
1182,110100,*MISSING*,MAIN CONCENTRATION,101101,Assets,Cash,Cash,Cash - Master Cash,0.35
1183,110105,*MISSING*,CENTRAL DEPOSITORY,101102,Assets,Cash,Cash,Cash - Depository,0.79
1184,110106,*MISSING*,B OF A LOCKBOX,101107,Assets,Cash,Cash,Cash - Lockbox,0.69


# Seq2Seq Model

In [17]:
import string
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
import re

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import time
import math
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
#helper functions

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:
def preprocess_text(text):

    text = text.lower().strip()
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('\\s+', ' ', text)

    return text

def process_hierarchy_string(df, row_idx, num_levels, level_prefix):

    hierarchy = []
    for l in range(num_levels):
        level_label = level_prefix+str(l+1)
        level_text = df.loc[row_idx, level_label]
        if pd.isna(level_text): continue
        else: 
            level_text = preprocess_text(level_text)
            hierarchy.append(level_text)

    hierarchy_string = ' '.join(hierarchy)
    #all_words = word_tokenize(hierarchy_string)
    
    return hierarchy_string 

process_hierarchy_string(df_mars, row_idx=0, num_levels=4, level_prefix = 'T L')


'revenue trade expenditures revenue 3rd party revenue products 3rd party revenue products 3rd party domestic'

In [92]:
df_fan_filtered = df_fan[df_fan['Similarity'] != 1.00]
df_mars_filtered = df_mars[df_mars['Similarity'] != 1.00]
df_pel_filtered = df_pel[df_pel['Similarity'] != 1.00]
df_rnd_filtered = df_rnd[df_rnd['Similarity'] != 1.00]

fan_sources = df_fan_filtered.loc[:,'S L1':'Source Description']
mars_sources = df_mars_filtered.loc[:,'S L1':'Source Description']
pel_sources = df_pel_filtered.loc[:,'S L1':'Source Description']
rnd_sources = df_rnd_filtered.loc[:,'S L1': 'Source Description']
source_data = pd.concat([mars_sources, fan_sources, pel_sources, rnd_sources]).astype(str)

fan_targets = df_fan_filtered.loc[:,'T L1':'Target Description']
mars_targets = df_mars_filtered.loc[:,'T L1':'Target Description']
pel_targets = df_pel_filtered.loc[:,'T L1':'Target Description']
rnd_targets = df_rnd_filtered.loc[:,'T L1':'Target Description']
target_data = pd.concat([mars_targets, fan_targets, pel_targets, rnd_targets]).astype(str)

x_train, x_test, y_train, y_test = train_test_split(source_data, target_data, test_size=0.15, shuffle=True, random_state=12)
print(f'Training size: {len(x_train)}  \nTesting size: {len(x_test)}')

Training size: 9470  
Testing size: 1672


In [74]:
SOS_token = 0
EOS_token = 1

class Vocabulary:

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "UNKNOWN"}
        self.n_words = 3  # Count SOS, EOS, UNKNOWN

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [75]:
def create_pairs(x_data, y_data, source_length ='short', target_length='short'):

    x_data = x_data.reset_index(drop=True)
    y_data = y_data.reset_index(drop=True)

    sources, targets = [], []

    for i in range(len(x_data)):
        source_hierarchy = process_hierarchy_string(x_data, row_idx=i, num_levels=10, level_prefix = 'S L')
        target_hierarchy = process_hierarchy_string(y_data, row_idx=i, num_levels=4, level_prefix = 'T L')

        source_description = preprocess_text(x_data.loc[i, 'Source Description'])
        target_description = preprocess_text(y_data.loc[i, 'Target Description'])

        if source_length == 'short':
            source_text = source_description
        elif source_length == 'long':
            source_text = source_hierarchy + " " + source_description
        else:
            print("Error: unrecognized input for source_length, try 'short' or 'long'")
            return 0

        if target_length == 'short':
            target_text = target_description  
        elif target_length == 'long':
            target_text = target_hierarchy + " " + target_description
        else:
            print("Error: unrecognized input for target_length, try 'short' or 'long'")
            return 0

        sources.append(source_text)
        targets.append(target_text)

    pairs = list((zip(sources, targets)))
    
    return pairs

def create_vocabulary(set1, set2, x_data, y_data, source_length='short', target_length='short'):

    input_set = Vocabulary(set1)
    output_set = Vocabulary(set2)

    pairs = create_pairs(x_data, y_data, source_length, target_length)

    print("%s pairs" % len(pairs))
    for pair in pairs:
        input_set.add_sentence(pair[0])
        output_set.add_sentence(pair[1])

    print("Vocabulary:")
    print(f'\t{input_set.name}: {input_set.n_words}')
    print(f'\t{output_set.name}: {output_set.n_words}')

    return input_set, output_set, pairs

In [76]:
MAX_LENGTH = 100

class Encoder(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class Decoder(nn.Module):

    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.name = 'Decoder'
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoder(nn.Module):

    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoder, self).__init__()
        self.name = 'AttnDecoder'
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [77]:
def indexes_from_sentence(set, sentence):
    sentence_indexes = []
    unknown_word = None
    for word in sentence.split(' '):
        if word not in set.word2index:
            sentence_indexes.append(2)
            unknown_word = word
        else:
            sentence_indexes.append(set.word2index[word])
            
    return sentence_indexes, unknown_word


def tensor_from_sentence(set, sentence):

    indexes, unknown_word = indexes_from_sentence(set, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1), unknown_word


def tensors_from_pair(pair, input_set, output_set):

    input_tensor, unknown_word_input = tensor_from_sentence(input_set, pair[0])
    target_tensor, unknown_word_target = tensor_from_sentence(output_set, pair[1])
    return (input_tensor, target_tensor), unknown_word_input, unknown_word_target

In [78]:
teacher_forcing_ratio = 0.0

def run_iter(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for en in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[en], encoder_hidden)
        encoder_outputs[en] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for de in range(target_length):
            if(decoder.name == 'AttnDecoder'):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                
            loss += criterion(decoder_output, target_tensor[de])
            decoder_input = target_tensor[de]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for de in range(target_length):
            if(decoder.name == 'AttnDecoder'):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[de])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [79]:
def train_model(input_set, output_set, encoder, decoder, pairs, n_iters, print_every=1000, learning_rate=0.01):

    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(random.choice(pairs), input_set, output_set)[0] for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for i in range(1, n_iters + 1):
        training_pair = training_pairs[i - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = run_iter(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, i / n_iters),
                                         i, i / n_iters * 100, print_loss_avg))


In [80]:
def evaluate(input_set, output_set, encoder, decoder, sentence, max_length=MAX_LENGTH):
    
    with torch.no_grad():
        input_tensor, unknown_word = tensor_from_sentence(input_set, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for en in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[en], encoder_hidden)
            encoder_outputs[en] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for de in range(max_length):
            if(decoder.name == 'AttnDecoder'):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                    
                decoder_attentions[de] = decoder_attention.data

            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            topv, topi = decoder_output.data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_set.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:de + 1], unknown_word


def print_predictions_on_test(input_set, output_set, encoder, decoder, sources, targets, n=10, source_length='short', target_length='short'):
    for i in range(n):
        pairs = create_pairs(sources, targets, source_length, target_length)
        pair = random.choice(pairs)
        print(f'SOURCE: {pair[0]}')
        print(f'TARGET: {pair[1]}')
        output_words, _, unknown_word = evaluate(input_set, output_set, encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        if unknown_word != None:
            print(f'UNKNOWN WORD: {unknown_word}')
        print(f'PREDICTION: {output_sentence} \n')


def eval_on_test(input_set, output_set, encoder, decoder, sources, targets, source_length='short', target_length='short'):

    prediction_similarities_to_targets = []
    pairs = create_pairs(sources, targets, source_length, target_length)
    score = 0
    max_score = len(pairs)
    bleu_scores = []
    unknown_words = []

    for pair in pairs:
        target = pair[1]
        output_words, _, unknown_word = evaluate(input_set, output_set, encoder, decoder, pair[0])
        prediction = ' '.join(output_words[:-1])

        target_words = [word_tokenize(target)]
        prediction_words = word_tokenize(prediction)
        bleu_score = round(sentence_bleu(target_words, prediction_words, weights=(1,)), 7)
        bleu_scores.append(bleu_score)
        
        if prediction == target: score += 1
        if unknown_word != None: unknown_words.append(unknown_word)

        pred_similarity = fuzz.token_sort_ratio(target, prediction)/100
        prediction_similarities_to_targets.append(pred_similarity)

    match_accuracy = score/max_score    
    avg_pred_similarity_to_target = np.mean(prediction_similarities_to_targets)
    unique_unknowns = set(unknown_words)

    print(f'Bleu score average: {np.mean(bleu_scores):.2f} \
            \nprediction exact match accuracy: {match_accuracy*100:.2f}% \
            \naverage prediction similarity to target: {avg_pred_similarity_to_target*100:.2f}% \
            \nunknown words found: {len(unique_unknowns)}/{max_score}')

    return match_accuracy, avg_pred_similarity_to_target, unique_unknowns

## Short source / short target strings

In [95]:
input_set1, output_set1, pairs1 = create_vocabulary('legacy', 'new', x_train, y_train, source_length='short', target_length='short')
print(random.choice(pairs1))

9470 pairs
Vocabulary:
	legacy: 4520
	new: 1704
('tax penalties', 'income tax penalties national')


In [96]:
hidden_size = 256
encoder1 = Encoder(input_set1.n_words, hidden_size).to(device)
decoder1 = Decoder(hidden_size, output_set1.n_words).to(device)

train_model(input_set1, output_set1, encoder1, decoder1, pairs1, 10000, print_every=2000)

1m 33s (- 6m 15s) (2000 20%) 3.2807
3m 18s (- 4m 57s) (4000 40%) 3.1647
4m 59s (- 3m 19s) (6000 60%) 2.9820
6m 37s (- 1m 39s) (8000 80%) 2.7130
8m 19s (- 0m 0s) (10000 100%) 2.4427


In [97]:
results = eval_on_test(input_set1, output_set1, encoder1, decoder1, x_test, y_test, source_length='short', target_length='short')

Bleu score average: 0.36             
prediction exact match accuracy: 14.35%             
average prediction similarity to target: 53.91%             
unknown words found: 320/1672


In [98]:
print_predictions_on_test(input_set1, output_set1, encoder1, decoder1, x_test, y_test, n=5, source_length='short', target_length='short')

SOURCE: accruals display equipment miscellaneous 2
TARGET: accruals display equipment stock racks
PREDICTION: accrued <EOS> 

SOURCE: bank account local currency 13
TARGET: bank account in local currency 13
PREDICTION: bank account in local currency <EOS> 

SOURCE: lease cost leased trucks
TARGET: vehicle leasing
PREDICTION: accum equipment accum <EOS> 

SOURCE: customer returns domestic manual postings
TARGET: nonsaleable returns
PREDICTION: ap 3rd party <EOS> 

SOURCE: donations
TARGET: expenses donations
PREDICTION: accounts payable <EOS> 



## Long source / short target strings

In [103]:
input_set2, output_set2, pairs2 = create_vocabulary('legacy', 'new', x_train, y_train, source_length = 'long', target_length = 'short')
print(random.choice(pairs2))

9470 pairs
Vocabulary:
	legacy: 4860
	new: 1704
('missing nan nan nan nan nan nan nan nan nan accounts payable adjustments', 'accounts payable')


In [104]:
hidden_size = 256
encoder2 = Encoder(input_set2.n_words, hidden_size).to(device)
decoder2 = Decoder(hidden_size, output_set2.n_words).to(device)

train_model(input_set2, output_set2, encoder2, decoder2, pairs2, 10000, print_every=2000)

4m 58s (- 19m 54s) (2000 20%) 3.3230
10m 16s (- 15m 24s) (4000 40%) 3.1819
16m 21s (- 10m 54s) (6000 60%) 2.8605
22m 18s (- 5m 34s) (8000 80%) 2.4186
28m 7s (- 0m 0s) (10000 100%) 2.1936


In [106]:
results = eval_on_test(input_set2, output_set2, encoder2, decoder2, x_test, y_test, source_length='long', target_length='short')

Bleu score average: 0.46             
prediction exact match accuracy: 19.86%             
average prediction similarity to target: 61.75%             
unknown words found: 323/1672


In [107]:
print_predictions_on_test(input_set2, output_set2, encoder2, decoder2, x_test, y_test, n=5, source_length='long', target_length='short')

SOURCE: m003 mars financial statement v2 full trial balan 72 net financing 59 total treasury liabilities 53 treasury debt 51 associate benefit reserves 615 pension reserve nan nan nan nan pension reserves mars gmbh pension aoci offset
TARGET: pension liabilities plan 14 noncurrent
PREDICTION: pension liabilities plan 1 noncurrent <EOS> 

SOURCE: m003 mars financial statement v2 full trial balan 31 net operating assets 18 total operating assets 12 gross operating assets 8 current operating assets 3 accounts receivable affil 139 accounts receivable affiliates nan nan nan ar inter company netting
TARGET: ar affiliate netting
PREDICTION: ap 3rd party domestic <EOS> 

SOURCE: m003 mars financial statement v2 full trial balan non corporate accounts profit and loss nan nan nan nan nan nan nan legal analytical ledger 117
TARGET: analytical ledger revenueexpense legal adjust
UNKNOWN WORD: 117
PREDICTION: analytical ledger revenueexpense legal adjust <EOS> 

SOURCE: mfps balance sheet mars fpps 

## Long source / long target strings

In [108]:
input_set3, output_set3, pairs3 = create_vocabulary('legacy', 'new', x_train, y_train, source_length = 'long', target_length = 'long')
print(random.choice(pairs3))

9470 pairs
Vocabulary:
	legacy: 4860
	new: 1763
('selling general and admin expenses selling general and admin expenses internet commissions nan nan nan nan nan nan nan 2601royalty payable minimum guarentee', 'selling general and admin expenses selling general and admin expenses internet commissions nan expenses royalty payable minimum guarantee')


In [109]:
hidden_size = 256
encoder3 = Encoder(input_set3.n_words, hidden_size).to(device)
decoder3 = Decoder(hidden_size, output_set3.n_words).to(device)

train_model(input_set3, output_set3, encoder3, decoder3, pairs3, 10000, print_every=2000)

8m 4s (- 32m 16s) (2000 20%) 2.7968
16m 38s (- 24m 57s) (4000 40%) 2.0575
25m 13s (- 16m 48s) (6000 60%) 1.6763
34m 19s (- 8m 34s) (8000 80%) 1.4124
42m 43s (- 0m 0s) (10000 100%) 1.2865


In [110]:
results = eval_on_test(input_set3, output_set3, encoder3, decoder3, x_test, y_test, source_length='long', target_length='long')

Bleu score average: 0.70             
prediction exact match accuracy: 14.00%             
average prediction similarity to target: 78.11%             
unknown words found: 323/1672


In [111]:
print_predictions_on_test(input_set3, output_set3, encoder3, decoder3, x_test, y_test, n=5, source_length='long', target_length='long')

SOURCE: m003 mars financial statement v2 full trial balan non corporate accounts balance sheet nan nan nan nan nan nan nan accruals currency results profit legal
TARGET: statutory future use statutory current and noncurrent liabilities statutory current and noncurrent liabilities statutory other liabilities current current liabilities oim legal adjustment
PREDICTION: statutory future use statutory current and noncurrent liabilities statutory current and noncurrent liabilities statutory property liabilities current current legal current liabilities <EOS> 

SOURCE: selling general and admin expenses selling general and admin expenses contra labor nan nan nan nan nan nan nan 4006contra labor
TARGET: selling general and admin expenses selling general and admin expenses contra labor nan expenses contra labor
PREDICTION: selling general and admin expenses selling general and admin expenses internet commissions nan expenses royalty <EOS> 

SOURCE: mfps balance sheet mars fpps structure 2926 n